# Phase 5: Deployment
## Movie Recommendation System - Streamlit App

**Objectives:**
1. Create a Streamlit web application
2. Load trained models (SVD, Content-Based, Hybrid)
3. Provide interactive movie recommendations
4. Display movie information and explanations

**Features:**
- User selection (from existing users)
- Multiple recommendation methods
- Movie details and genres
- Explanation of why movies are recommended

---
## 1. Create Project Structure

In [1]:
import os

# Define paths
PROJECT_PATH = 'D:/Courses/DL INTERNSHIP/THIRD PROJECT'
APP_PATH = f'{PROJECT_PATH}/app'

# Create app directory
os.makedirs(APP_PATH, exist_ok=True)

print(f"App directory: {APP_PATH}")

App directory: D:/Courses/DL INTERNSHIP/THIRD PROJECT/app


---
## 2. Create Streamlit App

In [2]:
streamlit_app = '''
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import load_npz
import os

# ===========================================
# CONFIGURATION
# ===========================================
st.set_page_config(
    page_title="Movie Recommender",
    page_icon="🎬",
    layout="wide"
)

# Paths
BASE_PATH = os.path.dirname(os.path.abspath(__file__))
DATA_PATH = os.path.join(BASE_PATH, "..", "data", "ml_ready")
MODELS_PATH = os.path.join(BASE_PATH, "..", "models")

# ===========================================
# LOAD DATA AND MODELS (Cached)
# ===========================================
@st.cache_data
def load_data():
    """Load preprocessed data."""
    # Load mappings
    with open(os.path.join(DATA_PATH, "mappings.pkl"), "rb") as f:
        mappings = pickle.load(f)
    
    # Load stats
    with open(os.path.join(DATA_PATH, "stats.pkl"), "rb") as f:
        stats = pickle.load(f)
    
    # Load movies
    movies_df = pd.read_parquet(os.path.join(DATA_PATH, "movies_train.parquet"))
    
    # Load train data for user history
    train_df = pd.read_parquet(os.path.join(DATA_PATH, "train.parquet"))
    
    # Load eval data
    with open(os.path.join(DATA_PATH, "eval_data.pkl"), "rb") as f:
        eval_data = pickle.load(f)
    
    return mappings, stats, movies_df, train_df, eval_data

@st.cache_resource
def load_models():
    """Load trained models."""
    # Load SVD model
    with open(os.path.join(MODELS_PATH, "svd_model.pkl"), "rb") as f:
        svd_model = pickle.load(f)
    
    # Load item similarity
    item_similarity = np.load(os.path.join(MODELS_PATH, "item_similarity.npy"))
    
    # Load hybrid config
    with open(os.path.join(MODELS_PATH, "hybrid_config.pkl"), "rb") as f:
        hybrid_config = pickle.load(f)
    
    return svd_model, item_similarity, hybrid_config

# ===========================================
# RECOMMENDATION FUNCTIONS
# ===========================================
def get_user_history(user_idx, train_df, movies_df):
    """Get movies rated by a user."""
    user_ratings = train_df[train_df["user_idx"] == user_idx].copy()
    user_ratings = user_ratings.merge(movies_df[["item_idx", "title", "genres"]], on="item_idx")
    user_ratings = user_ratings.sort_values("rating", ascending=False)
    return user_ratings[["title", "genres", "rating"]]

def predict_svd(user_idx, item_idx, svd_model):
    """Predict rating using SVD."""
    user_factors = svd_model["user_factors"]
    item_factors = svd_model["item_factors"]
    global_mean = svd_model["global_mean"]
    user_bias = svd_model["user_bias"]
    item_bias = svd_model["item_bias"]
    
    user_idx = int(user_idx)
    item_idx = int(item_idx)
    
    if user_idx < len(user_factors) and item_idx < len(item_factors):
        latent_pred = np.dot(user_factors[user_idx], item_factors[item_idx])
    else:
        latent_pred = 0
    
    u_bias = user_bias.get(user_idx, 0)
    i_bias = item_bias.get(item_idx, 0)
    
    pred = global_mean + u_bias + i_bias + latent_pred
    return np.clip(pred, 0.5, 5.0)

def get_svd_recommendations(user_idx, svd_model, user_positive, n_items, k=10):
    """Get SVD recommendations."""
    user_factors = svd_model["user_factors"]
    item_factors = svd_model["item_factors"]
    global_mean = svd_model["global_mean"]
    user_bias = svd_model["user_bias"]
    item_bias = svd_model["item_bias"]
    
    user_idx = int(user_idx)
    seen = user_positive.get(user_idx, set())
    
    # Compute predictions for all items
    if user_idx < len(user_factors):
        preds = np.dot(user_factors[user_idx], item_factors.T)
    else:
        preds = np.zeros(n_items)
    
    # Add biases
    preds += global_mean
    preds += user_bias.get(user_idx, 0)
    for i in range(len(preds)):
        preds[i] += item_bias.get(i, 0)
    
    # Mask seen items
    for item in seen:
        if item < len(preds):
            preds[item] = -np.inf
    
    # Get top-k
    top_k = np.argsort(preds)[-k:][::-1]
    scores = preds[top_k]
    
    return list(zip(top_k.tolist(), scores.tolist()))

def get_content_recommendations(user_idx, user_positive, item_similarity, 
                                 user_item_ratings, relevance_threshold=4.0, k=10):
    """Get content-based recommendations."""
    user_idx = int(user_idx)
    seen = user_positive.get(user_idx, set())
    n_items = item_similarity.shape[0]
    
    if len(seen) == 0:
        return []
    
    # Get liked items
    liked_items = []
    for item_idx in seen:
        if item_idx in user_item_ratings.get(user_idx, {}):
            if user_item_ratings[user_idx][item_idx] >= relevance_threshold:
                liked_items.append(item_idx)
    
    if len(liked_items) == 0:
        liked_items = list(seen)[:10]
    
    # Compute scores
    scores = np.zeros(n_items)
    for liked in liked_items:
        if liked < n_items:
            scores += item_similarity[liked]
    scores /= max(len(liked_items), 1)
    
    # Mask seen
    for item in seen:
        if item < n_items:
            scores[item] = -np.inf
    
    # Get top-k
    top_k = np.argsort(scores)[-k:][::-1]
    top_scores = scores[top_k]
    
    return list(zip(top_k.tolist(), top_scores.tolist()))

def get_hybrid_recommendations(user_idx, svd_model, user_positive, item_similarity,
                                user_item_ratings, n_items, alpha=0.5, k=10):
    """Get hybrid recommendations."""
    user_idx = int(user_idx)
    seen = user_positive.get(user_idx, set())
    n_items_local = min(n_items, item_similarity.shape[0])
    
    # SVD scores
    user_factors = svd_model["user_factors"]
    item_factors = svd_model["item_factors"]
    global_mean = svd_model["global_mean"]
    user_bias_dict = svd_model["user_bias"]
    item_bias_dict = svd_model["item_bias"]
    
    if user_idx < len(user_factors):
        svd_preds = np.dot(user_factors[user_idx], item_factors[:n_items_local].T)
    else:
        svd_preds = np.zeros(n_items_local)
    
    svd_preds += global_mean + user_bias_dict.get(user_idx, 0)
    for i in range(n_items_local):
        svd_preds[i] += item_bias_dict.get(i, 0)
    
    svd_scores = (svd_preds - 0.5) / 4.5  # Normalize to 0-1
    
    # Content-based scores
    liked_items = []
    for item_idx in seen:
        if item_idx in user_item_ratings.get(user_idx, {}):
            if user_item_ratings[user_idx][item_idx] >= 4.0:
                liked_items.append(item_idx)
    
    if len(liked_items) == 0:
        liked_items = [item for item in seen if item < n_items_local][:10]
    
    cb_scores = np.zeros(n_items_local)
    if len(liked_items) > 0:
        for liked in liked_items:
            if liked < n_items_local:
                cb_scores += item_similarity[liked, :n_items_local]
        cb_scores /= len(liked_items)
    
    # Combine
    hybrid_scores = alpha * svd_scores + (1 - alpha) * cb_scores
    
    # Mask seen
    for item in seen:
        if item < n_items_local:
            hybrid_scores[item] = -np.inf
    
    # Get top-k
    top_k = np.argsort(hybrid_scores)[-k:][::-1]
    top_scores = hybrid_scores[top_k]
    
    return list(zip(top_k.tolist(), top_scores.tolist()))

def get_popularity_recommendations(user_positive, item_popularity, user_idx, k=10):
    """Get popularity-based recommendations."""
    user_idx = int(user_idx)
    seen = user_positive.get(user_idx, set())
    
    # Sort by popularity
    sorted_items = sorted(item_popularity.items(), key=lambda x: x[1], reverse=True)
    
    recs = []
    for item_idx, pop in sorted_items:
        if item_idx not in seen:
            recs.append((item_idx, pop))
            if len(recs) >= k:
                break
    
    return recs

# ===========================================
# MAIN APP
# ===========================================
def main():
    st.title("🎬 Movie Recommendation System")
    st.markdown("*Powered by Matrix Factorization + Content-Based Filtering*")
    
    # Load data and models
    with st.spinner("Loading data and models..."):
        mappings, stats, movies_df, train_df, eval_data = load_data()
        svd_model, item_similarity, hybrid_config = load_models()
    
    n_users = mappings["n_users"]
    n_items = mappings["n_items"]
    user_positive = eval_data["user_positive_items"]
    item_popularity = stats["item_popularity"]
    
    # Build user-item ratings lookup
    user_item_ratings = {}
    for _, row in train_df.iterrows():
        u = int(row["user_idx"])
        i = int(row["item_idx"])
        r = row["rating"]
        if u not in user_item_ratings:
            user_item_ratings[u] = {}
        user_item_ratings[u][i] = r
    
    # Sidebar
    st.sidebar.header("⚙️ Settings")
    
    # User selection
    user_idx = st.sidebar.number_input(
        "User ID", 
        min_value=0, 
        max_value=n_users-1, 
        value=0,
        help="Select a user ID to get personalized recommendations"
    )
    
    # Recommendation method
    method = st.sidebar.selectbox(
        "Recommendation Method",
        ["Hybrid (SVD + Content)", "SVD (Collaborative)", "Content-Based (Genres)", "Popularity"]
    )
    
    # Number of recommendations
    n_recs = st.sidebar.slider("Number of Recommendations", 5, 20, 10)
    
    # Alpha for hybrid
    if method == "Hybrid (SVD + Content)":
        alpha = st.sidebar.slider(
            "Hybrid Alpha (SVD weight)", 
            0.0, 1.0, 
            hybrid_config["best_alpha"],
            help="0 = Pure Content-Based, 1 = Pure SVD"
        )
    else:
        alpha = 0.5
    
    # Main content
    col1, col2 = st.columns([1, 2])
    
    with col1:
        st.subheader(f"👤 User {user_idx} History")
        user_history = get_user_history(user_idx, train_df, movies_df)
        if len(user_history) > 0:
            st.write(f"Movies rated: {len(user_history)}")
            st.dataframe(
                user_history.head(10),
                use_container_width=True,
                hide_index=True
            )
        else:
            st.warning("No rating history found for this user.")
    
    with col2:
        st.subheader(f"🎯 Recommendations ({method})")
        
        # Get recommendations
        if method == "Hybrid (SVD + Content)":
            recs = get_hybrid_recommendations(
                user_idx, svd_model, user_positive, item_similarity,
                user_item_ratings, n_items, alpha=alpha, k=n_recs
            )
        elif method == "SVD (Collaborative)":
            recs = get_svd_recommendations(
                user_idx, svd_model, user_positive, n_items, k=n_recs
            )
        elif method == "Content-Based (Genres)":
            recs = get_content_recommendations(
                user_idx, user_positive, item_similarity,
                user_item_ratings, k=n_recs
            )
        else:  # Popularity
            recs = get_popularity_recommendations(
                user_positive, item_popularity, user_idx, k=n_recs
            )
        
        # Display recommendations
        if len(recs) > 0:
            rec_data = []
            for item_idx, score in recs:
                movie_info = movies_df[movies_df["item_idx"] == item_idx]
                if len(movie_info) > 0:
                    rec_data.append({
                        "Title": movie_info["title"].values[0],
                        "Genres": movie_info["genres"].values[0],
                        "Score": f"{score:.3f}" if method != "Popularity" else f"{int(score):,} ratings"
                    })
            
            rec_df = pd.DataFrame(rec_data)
            st.dataframe(rec_df, use_container_width=True, hide_index=True)
        else:
            st.warning("No recommendations available.")
    
    # Footer
    st.markdown("---")
    st.markdown(
        """
        **About this system:**
        - **SVD**: Learns latent factors from user-item interactions
        - **Content-Based**: Recommends movies with similar genres to what you liked
        - **Hybrid**: Combines both approaches for better recommendations
        - **Popularity**: Recommends most popular movies you haven\'t seen
        
        Built with MovieLens 32M dataset (30% sample) | SVD RMSE: 0.87
        """
    )

if __name__ == "__main__":
    main()
'''

# Write the Streamlit app
with open(f'{APP_PATH}/app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_app)

print(f"Streamlit app created: {APP_PATH}/app.py")

Streamlit app created: D:/Courses/DL INTERNSHIP/THIRD PROJECT/app/app.py


---
## 3. Create Requirements File

In [3]:
requirements = '''streamlit>=1.28.0
pandas>=2.0.0
numpy>=1.24.0
scipy>=1.10.0
pyarrow>=12.0.0
'''

with open(f'{APP_PATH}/requirements.txt', 'w') as f:
    f.write(requirements)

print(f"Requirements file created: {APP_PATH}/requirements.txt")
print("\nContents:")
print(requirements)

Requirements file created: D:/Courses/DL INTERNSHIP/THIRD PROJECT/app/requirements.txt

Contents:
streamlit>=1.28.0
pandas>=2.0.0
numpy>=1.24.0
scipy>=1.10.0
pyarrow>=12.0.0



---
## 4. Create README

In [4]:
readme = '''# 🎬 Movie Recommendation System

A hybrid movie recommendation system built with Matrix Factorization (SVD) and Content-Based Filtering.

## Features

- **SVD (Collaborative Filtering)**: Learns user preferences from rating patterns
- **Content-Based Filtering**: Recommends movies with similar genres
- **Hybrid Model**: Combines both approaches for better recommendations
- **Popularity Baseline**: Recommends trending movies

## Dataset

- MovieLens 32M Dataset (30% sample)
- ~9.6 million ratings
- ~60,000 users
- ~27,500 movies
- 19 genres

## Model Performance

| Model | RMSE | NDCG@10 |
|-------|------|--------|
| Global Mean | 1.05 | - |
| User-Item Bias | 0.92 | - |
| SVD | **0.87** | 0.015 |
| Hybrid | - | **0.040** |

## Installation

```bash
# Install dependencies
pip install -r requirements.txt

# Run the app
streamlit run app.py
```

## Project Structure

```
THIRD PROJECT/
├── app/
│   ├── app.py              # Streamlit application
│   ├── requirements.txt    # Python dependencies
│   └── README.md           # This file
├── data/
│   ├── processed/          # Cleaned data
│   └── ml_ready/           # ML-ready data
├── models/
│   ├── svd_model.pkl       # Trained SVD model
│   ├── item_similarity.npy # Genre similarity matrix
│   └── hybrid_config.pkl   # Hybrid model config
└── Notebooks/
    ├── 01_Data_Understanding_Cleaning_Sampling.ipynb
    ├── 02_EDA.ipynb
    ├── 03_ML_Preprocessing.ipynb
    └── 04_Model_Training.ipynb
```

## Usage

1. Select a User ID from the sidebar
2. Choose a recommendation method
3. Adjust the number of recommendations
4. For Hybrid mode, adjust the alpha slider:
   - α = 1.0: Pure SVD
   - α = 0.0: Pure Content-Based
   - α = 0.5: Equal weight

## Technologies

- Python 3.12
- Streamlit (Web UI)
- NumPy/SciPy (Matrix operations)
- Pandas (Data handling)
- Scikit-learn (Similarity computation)

## Author

Built as part of Deep Learning Internship - Third Project
'''

with open(f'{APP_PATH}/README.md', 'w', encoding='utf-8') as f:
    f.write(readme)

print(f"README created: {APP_PATH}/README.md")

README created: D:/Courses/DL INTERNSHIP/THIRD PROJECT/app/README.md


---
## 5. Create Run Script

In [5]:
# Windows batch file
run_bat = '''@echo off
echo Starting Movie Recommendation System...
echo.
echo Make sure you have installed the requirements:
echo   pip install -r requirements.txt
echo.
streamlit run app.py
pause
'''

with open(f'{APP_PATH}/run.bat', 'w') as f:
    f.write(run_bat)

print(f"Run script created: {APP_PATH}/run.bat")

Run script created: D:/Courses/DL INTERNSHIP/THIRD PROJECT/app/run.bat


---
## 6. Verify All Files

In [6]:
print("=" * 60)
print("APP FILES CREATED")
print("=" * 60)

for f in os.listdir(APP_PATH):
    filepath = os.path.join(APP_PATH, f)
    size = os.path.getsize(filepath) / 1024
    print(f"  {f}: {size:.1f} KB")

print("\n" + "=" * 60)
print("TO RUN THE APP")
print("=" * 60)
print(f"""  
1. Open a terminal/command prompt
2. Navigate to: {APP_PATH}
3. Install requirements:
   pip install -r requirements.txt
4. Run the app:
   streamlit run app.py
5. Open browser at: http://localhost:8501
""")

APP FILES CREATED
  app.py: 12.4 KB
  README.md: 2.1 KB
  requirements.txt: 0.1 KB
  run.bat: 0.2 KB

TO RUN THE APP
  
1. Open a terminal/command prompt
2. Navigate to: D:/Courses/DL INTERNSHIP/THIRD PROJECT/app
3. Install requirements:
   pip install -r requirements.txt
4. Run the app:
   streamlit run app.py
5. Open browser at: http://localhost:8501



---
## Phase 5 Summary

In [7]:
print("=" * 70)
print("PHASE 5 SUMMARY: DEPLOYMENT")
print("=" * 70)

print("\n" + "-" * 70)
print("STREAMLIT APP FEATURES")
print("-" * 70)
print("  ✓ User selection (60,284 users available)")
print("  ✓ Multiple recommendation methods:")
print("    - Hybrid (SVD + Content-Based)")
print("    - SVD (Collaborative Filtering)")
print("    - Content-Based (Genre Similarity)")
print("    - Popularity Baseline")
print("  ✓ Adjustable parameters (alpha, n_recs)")
print("  ✓ User rating history display")
print("  ✓ Movie details (title, genres, score)")

print("\n" + "-" * 70)
print("FILES CREATED")
print("-" * 70)
print(f"  Location: {APP_PATH}")
print("  - app.py: Main Streamlit application")
print("  - requirements.txt: Python dependencies")
print("  - README.md: Documentation")
print("  - run.bat: Windows launch script")

print("\n" + "-" * 70)
print("REQUIRED MODEL FILES")
print("-" * 70)
print("  From models/ folder:")
print("  - svd_model.pkl (17.75 MB)")
print("  - item_similarity.npy (2884.45 MB)")
print("  - hybrid_config.pkl")
print("  ")
print("  From data/ml_ready/ folder:")
print("  - mappings.pkl")
print("  - stats.pkl")
print("  - movies_train.parquet")
print("  - train.parquet")
print("  - eval_data.pkl")

print("\n" + "=" * 70)
print("PROJECT COMPLETE!")
print("=" * 70)
print("""
Congratulations! You have completed the Movie Recommendation System project.

Summary of all phases:
  Phase 1: Data Understanding, Cleaning & Sampling
  Phase 2: Exploratory Data Analysis (EDA)
  Phase 3: ML Preprocessing
  Phase 4: Model Training
  Phase 5: Deployment (Streamlit App)

To run the app:
  cd {}
  streamlit run app.py
""".format(APP_PATH))

PHASE 5 SUMMARY: DEPLOYMENT

----------------------------------------------------------------------
STREAMLIT APP FEATURES
----------------------------------------------------------------------
  ✓ User selection (60,284 users available)
  ✓ Multiple recommendation methods:
    - Hybrid (SVD + Content-Based)
    - SVD (Collaborative Filtering)
    - Content-Based (Genre Similarity)
    - Popularity Baseline
  ✓ Adjustable parameters (alpha, n_recs)
  ✓ User rating history display
  ✓ Movie details (title, genres, score)

----------------------------------------------------------------------
FILES CREATED
----------------------------------------------------------------------
  Location: D:/Courses/DL INTERNSHIP/THIRD PROJECT/app
  - app.py: Main Streamlit application
  - requirements.txt: Python dependencies
  - README.md: Documentation
  - run.bat: Windows launch script

----------------------------------------------------------------------
REQUIRED MODEL FILES
------------------------